# Getting started with Amazon Braket Hybrid Jobs

This tutorial shows how to run your first Amazon Braket Hybrid Job. To get started, we consider small circuits with only one qubit and one gate.

## Specify AWS region
For this demo, we submit the job to Braket-Gamma. To achieve this we set up an AWS session with Gamma endpoint.

In [6]:
import boto3
from braket.aws import AwsQuantumJob, AwsSession

region_name = "eu-west-2"
# endpoint_url = "https://braket-gamma.eu-west-2.amazonaws.com"
# endpoint_url = "https://braket.eu-west-2.amazonaws.com"
endpoint_url = "https://5sk8cnl96i.execute-api.eu-west-2.amazonaws.com/Prod"

# region_name = "us-west-1"
# endpoint_url = "https://braket-gamma.us-west-1.amazonaws.com"
# endpoint_url = "https://braket.us-west-1.amazonaws.com"

braket_client = boto3.client("braket", region_name=region_name, endpoint_url=endpoint_url)
aws_session = AwsSession(braket_client=braket_client)

In [7]:
# select which device to use
# device_arn="arn:aws:braket:::device/quantum-simulator/amazon/sv1"
device_arn="arn:aws:braket:eu-west-2::device/qpu/oqc/Lucy"
# device_arn = "arn:aws:braket:::device/qpu/rigetti/Aspen-11"

## Creating your first job

Once the script is finalized, you can create a Braket Job with `AwsQuantumJob`. When the job is created, Amazon Braket starts the job instance (based on EC2) and spins up a Docker container to run your algorithm script. Other configurations can be specified via keyword arguments. See the [developer guide](https://docs.aws.amazon.com/braket/latest/developerguide/what-is-braket.html) and other example notebooks to learn more about how to customize your jobs

This example uses the following inputs for `AwsQuantumJob`:
- <b>device</b>: The ARN of the Braket simulator or QPU to use in the job. It will be passed as an environment variable to the algorithm script. 
- <b>source_module</b>: The path to a file or a Python module that contains your algorithm script. It will be uploaded to the container for running the Braket Job.
- <b>wait_until_complete (optional)</b>: If True, the function call will wait until the Braket Job is completed and will additionally print logs to the local console. Otherwise, it will run asynchronously. The default is False.

In [8]:
%%time
# This cell should take about 5 mins
job = AwsQuantumJob.create(
    device=device_arn,
    source_module="script",
    entry_point="script.algorithm_script",
    wait_until_complete=True,
    aws_session=aws_session
)

Initializing Braket Job: arn:aws:braket:eu-west-2:537332306153:job/braket-job-default-1645644889268
............................................
2022-02-23 19:38:57,896 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-02-23 19:38:59,326 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-02-23 19:38:59,338 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-02-23 19:38:59,348 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {},
    "current_host": "algo-1",
    "framework_module": null,
    "hosts": [
        "algo-1"
    ],
    "hyperparameters": {},
    "input_config_dir": "/opt/ml/input/config",
    "input_data_config": {},
    "input_dir": "/opt/ml/input",
    "is_master": true,
    "job_name": "d7c996b8-5f70-481a-a3dd-b3f20a851554",
    "log_level": 20,
    "master_hostname": "alg

Counter({'1': 54, '0': 46})
Counter({'1': 60, '0': 40})
Counter({'1': 59, '0': 41})
Counter({'1': 55, '0': 45})
Counter({'1': 51, '0': 49})
Test job completed!!!!!
Code Run Finished
2022-02-23 19:39:36,793 sagemaker-training-toolkit INFO     Reporting training SUCCESS
CPU times: user 1.46 s, sys: 181 ms, total: 1.64 s
Wall time: 5min 10s


## Checking job state and loading results

The status of a Braket Job can be checked by calling `job.state()`. The state will be one of "QUEUED", "RUNNING", "FAILED", "COMPLETED", "CANCELLING", or "CANCELLED". 

In [9]:
job.state()

'COMPLETED'

In [10]:
results = job.result()  # will return once job.state() = "COMPLETED", should be 6 minutes
print("counts: ", results["counts"])
print("angles: ", results["angles"])

counts:  [{'0': 46, '1': 54}, {'0': 40, '1': 60}, {'0': 41, '1': 59}, {'0': 45, '1': 55}, {'0': 49, '1': 51}]
angles:  [1.5707963267948966, 1.6964600329384885, 1.8849555921538759, 1.853539665617978, 1.7278759594743864]
